In [1]:
import torch as t
import sys
sys.path.append('../')
from utils import get_score
import glob
import torch as t 
import numpy as np
import json
import time


In [2]:
label_path =   '/home/a/code/pytorch/zhihu/ddd/labels.json'
test_data_path='/home/a/code/pytorch/zhihu/ddd/val.npz' 
index2qid = np.load(test_data_path)['index2qid'].item()
with open(label_path) as f: 
      labels_info = json.load(f)
qid2label = labels_info['d']
label2qid = labels_info['id2label']

In [3]:
# 不进行数据增强的模型的融合
files = glob.glob('../checkpoints/result/*val.pth')
r2=0
for file in files:
    if 'MultiModel' not in file:
        print file
        r2+=t.load(file)

../checkpoints/result/LSTMText0.4119_word_val.pth
../checkpoints/result/RCNN_0.4037_char_val.pth
../checkpoints/result/LSTMText0.4031_char_val.pth
../checkpoints/result/RCNN_0.4115_word_val.pth
../checkpoints/result/DeepText0.4103_word_val.pth
../checkpoints/result/FastText3_word_0.409160_val.pth
../checkpoints/result/CNNText_tmp0.4109_word_val.pth
../checkpoints/result/CNNText_tmp0.4024_char_val.pth


In [4]:
true_labels = [qid2label[index2qid[2999967-200000+ii]] for ii in range(len(r2))]

In [5]:
# 进行数据增强的模型的融合
files = glob.glob('../checkpoints/result/tmp/*val.pth')
r_aug=0
for file in files:
    if 'MultiModel' not in file and 'weight5' not in file:
        print file
        r_aug+=t.load(file)

../checkpoints/result/tmp/DeepText_word_0.410051_aug_val.pth
../checkpoints/result/tmp/RCNN0.41344_aug_word_val.pth
../checkpoints/result/tmp/FastText3_word_weight1_0.40853_val.pth
../checkpoints/result/tmp/inception0.41254_aug_word_val.pth
../checkpoints/result/tmp/MultiCNNTextBNDeep_aug_char_0.38738_val.pth
../checkpoints/result/tmp/RCNN0.39854_aug_char_val.pth
../checkpoints/result/tmp/LSTMText0.41368_aug_word_val.pth


In [6]:
# MultiModel模型的融合
files = glob.glob('../checkpoints/result/tmp/*val.pth')
r_multi=0
for file in files:
    if 'MultiModel' in file :
        print file
        r_multi+=t.load(file)

../checkpoints/result/tmp/MultiModelallfast_0.419088_aug_val.pth
../checkpoints/result/tmp/MultiModel_0.4171859_val.pth
../checkpoints/result/tmp/MultiModel2_0.42560_val.pth
../checkpoints/result/tmp/MultiModel_0.42169_val.pth
../checkpoints/result/tmp/MultiModelall_0.4235_aug_val.pth
../checkpoints/result/tmp/MultiModel2w2c_0.4213_val.pth
../checkpoints/result/tmp/MultiModelallfast_0.41652_aug_val.pth
../checkpoints/result/tmp/MultiModel_0.41987_word_val.pth


In [7]:
# weight=5的模型,其实没什么用
files = glob.glob('../checkpoints/result/tmp/*val.pth')
r_5=0
for file in files:
    if 'weight5' in file :
        print file
        r_5+=t.load(file)

../checkpoints/result/tmp/MultiCNNTextBNDeep_word_weight5_0.409409_val.pth
../checkpoints/result/tmp/LSTMText_0.41230_weight5_val.pth
../checkpoints/result/tmp/RCNN_0.41226_weight5_val.pth
../checkpoints/result/tmp/inception_0.409912_weight5_val.pth
../checkpoints/result/tmp/FastText3_word_weight5_0.407992_val.pth


In [8]:
# 上述4大类模型的融合,这权重是手动调的
tmp = (r2+r_aug*1.15+r_multi*13+r_5*0.15)
result=(tmp).topk(5,1)[1]

In [ ]:
# 手动删除一些不太重要的模型
# tmp = tmp-t.load('../checkpoints/result/DeepText0.4103_word_val.pth')
# tmp = tmp+t.load('../checkpoints/result/DeepText0.4103_word_val.pth')*0.2
# tmp = tmp-t.load('../checkpoints/result/tmp/LSTMText0.41368_aug_word_val.pth')*0.1
# tmp = tmp-t.load('../checkpoints/result/CNNText_tmp0.4024_char_val.pth')*0.05
# tmp = tmp-t.load('../checkpoints/result/tmp/MultiCNNTextBNDeep_word_weight5_0.409409_val.pth')*0.009


In [9]:
# 计算在验证集上的分数 0.43065  提交后在0.43565以上
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,ss = get_score(predict_label_and_marked_label_list)
score,ss

(0.4306354273126943, [118573, 66909, 43404, 30157, 22047])